<img src="https://ucfai.org/core/fa19/2019-11-13-deep-rl/deep-rl/banner.png">

<div class="col-12">
    <span class="btn btn-success btn-block">
        Meeting in-person? Have you signed in?
    </span>
</div>

<div class="col-12">
    <h1> Learning by Doing, This Time With Neural Networks </h1>
    <hr>
</div>

<div style="line-height: 2em;">
    <p>by: 
        <strong> Alec Kerrigan</strong>
        (<a href="https://github.com/ahkerrigan">@ahkerrigan</a>)
     on 2019-11-13</p>
</div>